In [57]:
-- 0) Create Database
DROP DATABASE IF EXISTS movies_6610545421;

Commands completed successfully

Total execution time: 00:00:00.055

In [58]:
CREATE DATABASE movies_6610545421;

Commands completed successfully

Total execution time: 00:00:00.005

In [59]:
USE movies_6610545421;

Commands completed successfully

Total execution time: 00:00:00

In [60]:
DROP TABLE IF EXISTS Rating;
DROP TABLE IF EXISTS Reviewer;
DROP TABLE IF EXISTS Movie;

CREATE TABLE Movie (
  mID INT PRIMARY KEY,
  title VARCHAR(100) NOT NULL,
  releaseYear INT NOT NULL,
  director VARCHAR(100) NOT NULL
);

CREATE TABLE Reviewer (
  rID INT PRIMARY KEY,
  name VARCHAR(100) NOT NULL
);

CREATE TABLE Rating (
  rID INT NOT NULL,
  mID INT NOT NULL,
  stars INT CHECK (stars BETWEEN 1 AND 5),
  ratingDate DATE NULL,
  PRIMARY KEY (rID, mID),
  FOREIGN KEY (rID) REFERENCES Reviewer(rID),
  FOREIGN KEY (mID) REFERENCES Movie(mID)
);


Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.076

In [61]:
-- 1) Sample data (8 movies, 8 reviewers, 13 ratings)
INSERT INTO Movie (mID, title, releaseYear, director) VALUES
(101,'Gone with the Wind',1939,'Victor Fleming'),
(102,'Star Wars',1977,'George Lucas'),
(103,'The Sound of Music',1965,'Robert Wise'),
(104,'E.T.',1982,'Steven Spielberg'),
(105,'Titanic',1997,'James Cameron'),
(106,'Snow White',1937,'David Hand'),
(107,'Avatar',2009,'James Cameron'),
(108,'Raiders of the Lost Ark',1981,'Steven Spielberg');


INSERT INTO Reviewer (rID,name) VALUES
(201,'Sarah Martinez'),
(202,'Daniel Lewis'),
(203,'Mike Anderson'),
(204,'Ashley White'),
(205,'James Lee'),
(206,'Elizabeth King'),
(207,'David Brown'),
(208,'Susan Wilson');

INSERT INTO Rating (rID,mID,stars,ratingDate) VALUES
(201,101,4,'2024-01-15'),
(201,102,5,'2024-02-12'),
(202,103,3,'2024-02-27'),
(203,104,4,'2024-03-03'),
(203,105,2,'2024-03-25'),
(204,105,4,'2024-04-02'),
(205,101,5,'2024-04-08'),
(205,106,3,'2024-05-10'),
(206,107,4,'2024-06-01'),
(206,108,5,'2024-06-18'),
(207,102,2, NULL),
(208,103,4,'2024-02-29'),
(208,107,5,'2024-06-22');

Commands completed successfully

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.009

In [62]:
-- Q1: Show each movie with its average rating and overall average
SELECT 
  m.title, m.director, m.releaseYear,
  AVG(r.stars*1.0) AS avg_movie_rating,
  (SELECT AVG(stars*1.0) FROM Rating) AS overall_avg_rating
FROM Movie m
JOIN Rating r ON m.mID = r.mID
GROUP BY m.mID, m.title, m.director, m.releaseYear;


(8 row(s) affected)

Total execution time: 00:00:01.007

title,director,releaseYear,avg_movie_rating,overall_avg_rating
Gone with the Wind,Victor Fleming,1939,4.50000,3.84615
Star Wars,George Lucas,1977,3.50000,3.84615
The Sound of Music,Robert Wise,1965,3.50000,3.84615
E.T.,Steven Spielberg,1982,4.00000,3.84615
Titanic,James Cameron,1997,3.00000,3.84615
Snow White,David Hand,1937,3.00000,3.84615
Avatar,James Cameron,2009,4.50000,3.84615
Raiders of the Lost Ark,Steven Spielberg,1981,5.00000,3.84615


In [63]:
-- Q2: All movies (even without ratings) + simple category
SELECT 
  m.title, m.releaseYear, m.director,
  CASE
    WHEN COUNT(r.stars) = 0          THEN 'Not Rated'
    WHEN AVG(r.stars) >= 4           THEN 'Highly Rated'
    WHEN AVG(r.stars) >= 3           THEN 'Well Rated'
    ELSE 'Poorly Rated'
  END AS rating_category
FROM Movie m
LEFT JOIN Rating r ON r.mID = m.mID
GROUP BY m.mID, m.title, m.releaseYear, m.director
ORDER BY m.title;


(8 row(s) affected)

Total execution time: 00:00:01.004

title,releaseYear,director,rating_category
Avatar,2009,James Cameron,Highly Rated
E.T.,1982,Steven Spielberg,Highly Rated
Gone with the Wind,1939,Victor Fleming,Highly Rated
Raiders of the Lost Ark,1981,Steven Spielberg,Highly Rated
Snow White,1937,David Hand,Well Rated
Star Wars,1977,George Lucas,Well Rated
The Sound of Music,1965,Robert Wise,Well Rated
Titanic,1997,James Cameron,Well Rated


In [64]:
-- Q3: All reviewers, number of ratings, days since last rating
SELECT 
  rv.name,
  COUNT(r.stars) AS total_ratings,
  CASE
    WHEN MAX(r.ratingDate) IS NULL THEN NULL
    ELSE DATEDIFF(NOW(), MAX(r.ratingDate))
  END AS days_since_last
FROM Reviewer rv
LEFT JOIN Rating r ON r.rID = rv.rID
GROUP BY rv.rID, rv.name
ORDER BY rv.name;


(8 row(s) affected)

Total execution time: 00:00:01.010

name,total_ratings,days_since_last
Ashley White,1,523
Daniel Lewis,1,558
David Brown,1,NULL
Elizabeth King,2,446
James Lee,2,485
Mike Anderson,2,531
Sarah Martinez,2,573
Susan Wilson,2,442


In [65]:
-- Q4: Movie–Reviewer pairs with status (rated / not rated)
SELECT 
  m.title,
  rv.name AS reviewer,
  r.stars,
  CASE WHEN r.stars IS NULL THEN 'Not Rated' ELSE 'Rated' END AS status
FROM Movie m
CROSS JOIN Reviewer rv
LEFT JOIN Rating r ON r.mID = m.mID AND r.rID = rv.rID
ORDER BY m.title, rv.name;


(64 row(s) affected)

Total execution time: 00:00:01.014

title,reviewer,stars,status
Avatar,Ashley White,NULL,Not Rated
Avatar,Daniel Lewis,NULL,Not Rated
Avatar,David Brown,NULL,Not Rated
Avatar,Elizabeth King,4,Rated
Avatar,James Lee,NULL,Not Rated
Avatar,Mike Anderson,NULL,Not Rated
Avatar,Sarah Martinez,NULL,Not Rated
Avatar,Susan Wilson,5,Rated
E.T.,Ashley White,NULL,Not Rated
E.T.,Daniel Lewis,NULL,Not Rated


In [66]:
-- Q5: Average rating and count by month (YYYY-MM)
SELECT 
  DATE_FORMAT(r.ratingDate, '%Y-%m') AS month,
  AVG(r.stars) AS avg_rating,
  COUNT(*) AS num_ratings
FROM Rating r
WHERE r.ratingDate IS NOT NULL
GROUP BY DATE_FORMAT(r.ratingDate, '%Y-%m')
ORDER BY month;


(6 row(s) affected)

Total execution time: 00:00:01.002

month,avg_rating,num_ratings
2024-01,4.0000,1
2024-02,4.0000,3
2024-03,3.0000,2
2024-04,4.5000,2
2024-05,3.0000,1
2024-06,4.6667,3


In [67]:
-- Q6: Reviewer average vs overall average (simple view)
SELECT 
  rv.name,
  AVG(r.stars) AS avg_reviewer_rating,
  (SELECT AVG(stars) FROM Rating) AS overall_avg
FROM Reviewer rv
JOIN Rating r ON r.rID = rv.rID
GROUP BY rv.rID, rv.name
ORDER BY avg_reviewer_rating DESC;


(8 row(s) affected)

Total execution time: 00:00:01.016

name,avg_reviewer_rating,overall_avg
Sarah Martinez,4.5000,3.8462
Elizabeth King,4.5000,3.8462
Susan Wilson,4.5000,3.8462
Ashley White,4.0000,3.8462
James Lee,4.0000,3.8462
Daniel Lewis,3.0000,3.8462
Mike Anderson,3.0000,3.8462
David Brown,2.0000,3.8462


In [68]:
-- Q7: Director performance , number of movies and avg rating
SELECT 
  m.director,
  COUNT(DISTINCT m.mID) AS num_movies,
  AVG(r.stars) AS avg_director_rating
FROM Movie m
JOIN Rating r ON r.mID = m.mID
GROUP BY m.director
ORDER BY avg_director_rating DESC, num_movies DESC;


(6 row(s) affected)

Total execution time: 00:00:01.011

director,num_movies,avg_director_rating
Steven Spielberg,2,4.5000
Victor Fleming,1,4.5000
James Cameron,2,3.7500
George Lucas,1,3.5000
Robert Wise,1,3.5000
David Hand,1,3.0000


In [69]:
-- Q8: Rating distribution buckets per movie
SELECT 
  m.title,
  r.stars,
  CASE
    WHEN r.stars <= 2 THEN 'Low'
    WHEN r.stars <= 4 THEN 'Medium'
    ELSE 'High'
  END AS rating_group
FROM Movie m
JOIN Rating r ON r.mID = m.mID
ORDER BY m.title, r.stars;


(13 row(s) affected)

Total execution time: 00:00:01.004

title,stars,rating_group
Avatar,4,Medium
Avatar,5,High
E.T.,4,Medium
Gone with the Wind,4,Medium
Gone with the Wind,5,High
Raiders of the Lost Ark,5,High
Snow White,3,Medium
Star Wars,2,Low
Star Wars,5,High
The Sound of Music,3,Medium


In [70]:
-- Q9: Monthly rating summary (count + average)
SELECT 
  DATE_FORMAT(r.ratingDate, '%Y-%m') AS month,
  COUNT(*) AS num_ratings,
  AVG(r.stars) AS avg_rating
FROM Rating r
WHERE r.ratingDate IS NOT NULL
GROUP BY DATE_FORMAT(r.ratingDate, '%Y-%m')
ORDER BY month;


(6 row(s) affected)

Total execution time: 00:00:01.006

month,num_ratings,avg_rating
2024-01,1,4.0000
2024-02,3,4.0000
2024-03,2,3.0000
2024-04,2,4.5000
2024-05,1,3.0000
2024-06,3,4.6667


In [71]:
-- Q10: Movies rated by reviewers who rated 3+ different movies
SELECT DISTINCT m.title
FROM Movie m
JOIN Rating r ON r.mID = m.mID
WHERE r.rID IN (
  SELECT rID
  FROM Rating
  GROUP BY rID
  HAVING COUNT(DISTINCT mID) >= 3
)
ORDER BY m.title;


(0 row(s) affected)

Total execution time: 00:00:01.011

title
